In [ ]:
import osmnx as ox
from pathlib import Path
import os
import geopandas as gpd

from mclights.utils.get_data import get_data_osmx_or_local
from mclights.utils.convert_coordinates import convert_to_points

project_dir = Path(os.getcwd()).parent
data_dir = project_dir / "data"
temp_data_dir = project_dir / "temp_data"
raw_data_dir = project_dir / "raw_data"

In [ ]:
names_combination = ["McDonald's", "McDonald´s", "McDonald’s"]

mcdonalds = get_data_osmx_or_local(
    raw_data_dir / "mcdonalds.gpkg",
    {"amenity": "fast_food", "name": names_combination}
)

# mcdonald = mcdonald.loc[mcdonald["name"].isin(possible_names), ["geometry"]]
# mcdonald = mcdonald[mcdonald["geometry"].notna()]


In [ ]:
temp = mcdonalds.copy()
temp = temp.reset_index(drop=True)
temp = temp[temp["name"].isin(names_combination)]
# temp = temp[["geometry", "addr:city"]]

temp["geometry"] = temp["geometry"].apply(convert_to_points)
temp.to_excel(temp_data_dir / "mc_donalds.xlsx")

In [ ]:
# IT'S RAW CODE FROM MY PREVIOS VERSION - DO NOT USE

def get_city(lat, lon):
    """Get the city name from latitude and longitude using reverse geocoding."""
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True, language="pl")
        if location:
            address = location.raw.get("address", {})
            return address.get("city") or address.get("town") or address.get("village")
    except GeocoderTimedOut:
        logger.warning("GeocoderTimeOut Error.")
        return None
    except GeocoderUnavailable:
        logger.warning("Something is wrong.")
        return None
    return None


mcdonald["LAT"], mcdonald["LON"] = zip(*mcdonald["geometry"].apply(get_coordinates), strict=False)
mcdonald = mcdonald.drop(columns=["geometry"])

mask_empty = mcdonald.isna().any(axis=1)
logger.info(f"Removed {mask_empty.sum()} rows that contain NaN values.")
mcdonald = mcdonald.loc[~mask_empty]

logger.info("Adding cities to a locations.")
geolocator = Nominatim(user_agent="mcdonalds_locator")
mcdonald["CITY"] = mcdonald.progress_apply(lambda row: get_city(row["LAT"], row["LON"]), axis=1)

mcdonald.to_csv(FILE_PATH, index=False)
